In [ ]:
def load_data(file):
    df = feather.read_dataframe(file)

    weekday_map = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
    df['weekday'] = df['week'].map(weekday_map)

    n_type = pd.unique(df['type'])
    n_date = pd.unique(df['date_time'])
    n_weekdays = pd.unique(df['weekday'])
    period = 48
    begin = "2012-02-27"
    end = "2012-03-12"
    mask = (df['date_time'] > begin) & (df['date_time'] <= end)
    data = df.loc[mask]

    data = data[df['type'] == n_type[0]]
    return data

data = load_data('DT_4_ind.dms')

data.plot(x='date_time', y='value')



period = 48
N = data.shape[0] # number of observations in the train set
window = N / period # number of days in the train set

weekly = data['weekday']
x = np.array(range(1, period+1))
daily = np.tile(x, int(window))

matrix_gam = pd.DataFrame(columns=['daily', 'weekly', 'load'])
matrix_gam['load'] = data['value']
matrix_gam['daily'] = daily
matrix_gam['weekly'] = weekly

data

matrix_gam[['daily','weekly']]

zDaily = np.array(matrix_gam['daily'])
zWeekly = np.array(matrix_gam['weekly'])
zload = np.array(matrix_gam['load'])
zFusion = np.array(matrix_gam[['daily','weekly']])

zFusion



gam = LinearGAM(n_splines=10).gridsearch(zFusion, zload)

## plotting
plt.figure();
fig, axs = plt.subplots(1,2);
fig.set_figheight(10)
fig.set_figwidth(15)

titles = ['daily', 'weekly']
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    ax.set_title(titles[i]);

predictions = gam.predict(matrix_gam[['daily', 'weekly']])

fig = plt.figure(figsize=(15, 10))
plt.plot(data['date_time'], matrix_gam['load'])
plt.plot(data['date_time'], predictions)
plt.xticks(rotation='vertical')
plt.legend(['True', 'Predicted'])

# ==========FIMGAMDOCARA=============